In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors

# Configuration: Toggle to include already purchased products
INCLUDE_PURCHASED_PRODUCTS = True


In [2]:
# Load the dataset
df = pd.read_csv("stockist_data.csv")

# Define the product columns
product_columns = [
    "AIS(Air Insulated Switchgear)", "RMU(Ring Main Unit)", "PSS(Compact Sub-Stations)",
    "VCU(Vacuum Contactor Units)", "E-House", "VCB(Vacuum Circuit Breaker)",
    "ACB(Air Circuit Breaker)", "MCCB(Moduled Case Circuit Breaker)",
    "SDF(Switch Disconnectors)", "BBT(Busbar Trunking)", "Modular Switches"
]

In [3]:
# Extract user-product matrix and convert to binary
user_product_matrix = df.set_index("Partner_id")[product_columns].astype(int)

# Save the user IDs separately
user_ids = user_product_matrix.index.tolist()

# Split data into train and test sets (80% train, 20% test)
train_data, test_data = train_test_split(user_product_matrix, test_size=0.2, random_state=42)


In [4]:
# Save to CSV
train_data.reset_index().to_csv("train_data.csv", index=False)
test_data.reset_index().to_csv("test_data.csv", index=False)

# Fit KNN model on training data
model = NearestNeighbors(metric="cosine", algorithm="brute")
model.fit(train_data)
train_user_ids = train_data.index.tolist()


In [5]:
# Fit KNN Model for User-Based Collaborative Filtering on training data
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(train_data)



NearestNeighbors(algorithm='brute', metric='cosine')

In [6]:
recommendations = []

for partner_id in test_data.index:
    partner_vector = test_data.loc[[partner_id]]  # Pass as DataFrame with column names
    distances, indices = model.kneighbors(partner_vector, n_neighbors=6)

    recommended_products = []
    similarity_list = []

    for idx, dist in zip(indices.flatten(), distances.flatten()):
        if idx == 0:
            continue  # skip self

        similarity = 1 - dist
        similarity_list.append(similarity)

        neighbor_id = train_user_ids[idx]
        neighbor_products = set(train_data.loc[neighbor_id][train_data.loc[neighbor_id] == 1].index)

        if INCLUDE_PURCHASED_PRODUCTS:
            recommended_products.extend(list(neighbor_products))
        else:
            purchased_products = set(test_data.loc[partner_id][test_data.loc[partner_id] == 1].index)
            recommended_products.extend(list(neighbor_products - purchased_products))

    recommended_products = list(dict.fromkeys(recommended_products))[:3]

    recommendations.append({
        "Partner_id": partner_id,
        "Recommended_Products": recommended_products,
        "Avg_Similarity_Score": round(np.mean(similarity_list[1:]), 6) if len(similarity_list) > 1 else 0
    })


In [7]:
# Save recommendations
recommendations_df = pd.DataFrame(recommendations)
recommendations_df.to_csv("User_Based_Recommendations.csv", index=False)
print("Recommendations saved to 'User_Based_Recommendations.csv'")


Recommendations saved to 'User_Based_Recommendations.csv'


Evaluation Code

In [25]:
#Import Required Libraries
import pandas as pd

In [26]:
# Load test data with one-hot encoded product columns
test_df = pd.read_csv("test_data.csv")

In [27]:
# Load recommendation output (Top-N recommendations per partner)
recommendations_df = pd.read_csv("User_Based_Recommendations.csv")

In [28]:
# Fix column naming inconsistency if needed
recommendations_df = recommendations_df.rename(columns={"Partner_ID": "Partner_id"})

In [29]:
#Identify Producy column
meta_cols = [
    'Partner_id', 'Stockist_Type', 'Scheme_Type', 'Sales_Value_Last_Period',
    'Sales_Quantity_Last_Period', 'MRP', 'Growth_Percentage', 'Discount_Applied',
    'Bulk_Purchase_Tendency', 'New_Stockist', 'Feedback_Score'
]
product_cols = [col for col in test_df.columns if col not in meta_cols]


In [30]:
#Convert Product Columns to Purchased List
# ------------------------------
test_df[product_cols] = test_df[product_cols].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

# Purchased product list per partner
test_df['Purchased_Products'] = test_df[product_cols].apply(
    lambda row: [prod for prod, val in zip(product_cols, row) if val == 1], axis=1
)

# Remove non-buyers
test_df['has_purchase'] = test_df['Purchased_Products'].apply(lambda x: len(x) > 0)


In [31]:
# Ensure list parsing from string if needed
recommendations_df["Recommended_Products"] = recommendations_df["Recommended_Products"].apply(eval)

# Merge with test dataset
df_all = pd.merge(
    test_df[['Partner_id', 'Purchased_Products', 'has_purchase']],
    recommendations_df[['Partner_id', 'Recommended_Products']],
    on='Partner_id',
    how='left'
)

In [32]:
# Fill any missing recommendations with empty list
df_all['Recommended_Products'] = df_all['Recommended_Products'].apply(
    lambda x: x if isinstance(x, list) else []
)

In [33]:
results = []

for k in [1, 2, 3]:
    precision_list = []
    recall_list = []

    for _, row in df_all.iterrows():
        actual_set = set(row["Purchased_Products"])
        recommended_k = row["Recommended_Products"][:k]

        if not actual_set:
            continue  # skip if no purchases

        tp = len(set(recommended_k) & actual_set)
        precision = tp / k
        recall = tp / len(actual_set)

        precision_list.append(precision)
        recall_list.append(recall)

    avg_precision = round(sum(precision_list) / len(precision_list), 4) if precision_list else 0
    avg_recall = round(sum(recall_list) / len(recall_list), 4) if recall_list else 0
    f1_k = round(2 * avg_precision * avg_recall / (avg_precision + avg_recall), 4) if (avg_precision + avg_recall) else 0

    results.append({        "Top-K": k,
        "Avg Precision": avg_precision,
            "Avg Recall": avg_recall,
        "Avg F1 Score": f1_k
    })


In [34]:
#Display Results

print("===== Top-K Recommendation Evaluation (Corrected) =====")
for r in results:
    print(f"\nTop-{r['Top-K']}")
    print(f"  Avg Precision : {r['Avg Precision']}")
    print(f"  Avg Recall    : {r['Avg Recall']}")
    print(f"  Avg F1 Score  : {r['Avg F1 Score']}")

===== Top-K Recommendation Evaluation (Corrected) =====

Top-1
  Avg Precision : 0.875
  Avg Recall    : 0.1758
  Avg F1 Score  : 0.2928

Top-2
  Avg Precision : 0.8675
  Avg Recall    : 0.3483
  Avg F1 Score  : 0.497

Top-3
  Avg Precision : 0.8783
  Avg Recall    : 0.5264
  Avg F1 Score  : 0.6583
